# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('./data/song_data')

In [5]:
# my own work
song_files[0]

'/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json'

In [6]:
filepath = '.'

In [7]:
df = pd.read_json(song_files[0], lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARXR32B1187FB57099,NaN,,NaN,Gob,209.60608,1,SOFSOCN12A8C143F5D,Face the Ashes,2007


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
artist_id           1 non-null object
artist_latitude     0 non-null float64
artist_location     1 non-null object
artist_longitude    0 non-null float64
artist_name         1 non-null object
duration            1 non-null float64
num_songs           1 non-null int64
song_id             1 non-null object
title               1 non-null object
year                1 non-null int64
dtypes: float64(3), int64(2), object(5)
memory usage: 160.0+ bytes


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
# my work - select columns
df_select = df[['song_id','title','artist_id', 'year', 'duration']]

In [10]:
# my work - df.values
df_select = df_select.values

In [11]:
# my work - index to select the first record
df_select = df_select[0]

In [12]:
# my work - convert array to a list
df_select = df_select.tolist()

In [13]:
# my work - show result
df_select

['SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, 209.60608]

In [14]:
song_data = df[['song_id','title','artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, 209.60608]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [15]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [16]:
artist_data = df[['artist_id','artist_name','artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARXR32B1187FB57099', 'Gob', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
log_files = get_files('./data/log_data')

In [19]:
filepath = './data/log_data'

In [20]:
df = pd.read_json(log_files[0], lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Kevin,M,0,Arellano,NaN,free,"Harrisburg-Carlisle, PA",GET,Home,1.540007e+12,514,None,200,1542069417796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,None,Logged In,Maia,F,0,Burke,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540677e+12,510,None,200,1542071524796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [21]:
# my work
df_nextsong = df[df.page == 'NextSong']

In [22]:
# my work
df_nextsong.ts = pd.to_datetime(df_nextsong.ts, unit='ms')

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [23]:
df_nextsong.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,2018-11-13 00:40:37.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


In [24]:
df = df[df.page == 'NextSong']
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


In [25]:
t = pd.to_datetime(df_nextsong.ts, unit='ms')
t.head()

1   2018-11-13 00:40:37.796
3   2018-11-13 01:12:29.796
4   2018-11-13 03:19:02.796
5   2018-11-13 03:51:52.796
8   2018-11-13 05:00:06.796
Name: ts, dtype: datetime64[ns]

In [26]:
time_data = [[t.dt.time], [t.dt.hour], [t.dt.day], [t.dt.weekofyear], [t.dt.month], [t.dt.year], [t.dt.weekday]]
column_labels = ['time', 'hour', 'day', 'weekofyear', 'month', 'day', 'weekday']

In [27]:
# my work
data = {'time': t.dt.time, 'hour':t.dt.hour, 'day':t.dt.day, 'weekofyear':t.dt.weekofyear,
        'month':t.dt.month, 'year':t.dt.year, 'weekday':t.dt.weekday}

In [28]:
time_df = pd.DataFrame(data=data)
time_df.head()

,time,hour,day,weekofyear,month,year,weekday
1,00:40:37.796000,0,13,46,11,2018,1
3,01:12:29.796000,1,13,46,11,2018,1
4,03:19:02.796000,3,13,46,11,2018,1
5,03:51:52.796000,3,13,46,11,2018,1
8,05:00:06.796000,5,13,46,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [29]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [30]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [31]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [46]:
# my testing
# count = 0
# for index, row in df.iterrows():
#     print(row.length)
#     count += 1
#     if count >5:
#         break



In [33]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
        #(songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)

    # insert songplay record
    songplay_data = (time_df.time,  user_df.userId, user_df.level, songid, artistid, )
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

IndexError: tuple index out of range

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.